In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from transformers import (
                        pipeline, 
                        AutoTokenizer, 
                        AutoModelForCausalLM
                        )
from datasets import Dataset
import torch
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from dotenv import load_dotenv
import warnings
import s3fs

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()
pd.set_option("display.max_columns", None)
warnings.simplefilter("ignore")
fs = s3fs.S3FileSystem(
            client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"},
            key=os.environ["Accesskey"],
            secret=os.environ["Secretkey"],
            token=os.environ["Token"]
)

In [4]:
with fs.open("elissamim/text_classification_men/data/stages-votes.json", "r") as file:
    df = pd.read_json(file)

df = df.groupby("phrase_text", as_index = False)["sol"].apply(lambda x: x.mode().iloc[0])
df["sol"]=df["sol"].apply(lambda x: 1 if x == "ok" else 0)

In [5]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
)

# Create a text generator
generator = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
)

def few_shot_classification(text:str):

    prompt = f"""
    Tu es un expert en orientation scolaire. Ta tâche est de déterminer si un intitulé de stage de 3ᵉ est acceptable pour un collégien. Réponds uniquement par "oui" ou "non".
    
    Exemples :

    Example 1:
    Intitulé : "Stage dans une mairie"  
    Acceptable ? oui

    Example 2:
    Intitulé : "Stage chez un tatoueur"  
    Acceptable ? non

    Example 3:
    Intitulé : "Stage d’observation dans un cabinet vétérinaire"  
    Acceptable ? oui

    Example 4:
    Intitulé : "Stage dans une boîte de nuit"  
    Acceptable ? non

    Example 5:
    Intitulé : "Aide à la comptabilité dans une brasserie artisanale"
    Acceptable ? oui

    Example 6:
    Intitulé : "Aide à un sommelier pour la dégistation et l'évaluation de vins"
    Acceptable ? non
    
    Maintenant, analyse le nouvel intitulé :
    
    Intitulé : "{text}"  
    Acceptable ?
    """

    response =  generator(
        prompt,
        max_new_tokens=10,
        do_sample=False,
        return_full_text=False
    )[0]["generated_text"]

    if "oui" in response.lower():
        return 1
    
    return 0

# We use a HuggingFace Dataset
hf=Dataset.from_pandas(df)
hf=hf.map(lambda x: {"sol_generated": few_shot_classification(x["phrase_text"])})

# We evaluate the model
print(f"Accuracy: {accuracy_score(hf["sol"], hf["sol_generated"])}")
print(f"Classification report: {classification_report(hf["sol"], hf["sol_generated"])}")
print(f"Confusion matrix: {confusion_matrix(hf["sol"], hf["sol_generated"])}")

Loading checkpoint shards: 100%|██████████| 2/2 [00:54<00:00, 27.33s/it]
Device set to use cpu
